In [3]:
from midiutil import MIDIFile

In [138]:
# As notas com sustenido
NOTES_FLAT = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
# As notas com bemol
NOTES_SHARP = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

def NoteToMidi(KeyOctave):
    # KeyOctave is formatted like 'C#3'
    # Fonte: https://stackoverflow.com/questions/13926280/musical-note-string-c-4-f-3-etc-to-midi-note-value-in-python
    key = KeyOctave[:-1]  # eg C, Db
    octave = KeyOctave[-1]   # eg 3, 4
    answer = -1

    try:
        if 'b' in key:
            pos = NOTES_FLAT.index(key)
        else:
            pos = NOTES_SHARP.index(key)
    except:
        print('The key is not valid', key)
        return answer

    answer += pos + 12 * (int(octave) + 1) + 1
    return answer

def MidiToNote(valor):
    '''Transformar um valor Midi a Nota'''
    return NOTES_SHARP[valor % 12] + str(valor//12 - 1)

def adicionarOitava(nota, oitava):
    '''Adiciona a oitava: ("Gb",3) --> Gb3 '''
    return nota + str(oitava)

print(f'C4 corresponde a : {NoteToMidi("C4")}')
print(f'62 corresponde a : {MidiToNote(62)}')

C4 corresponde a : 60
62 corresponde a : D4


# Questão 1

In [171]:
from midiutil import MIDIFile

# --------------------------
track    = 0  #faixa
channel  = 0  # canal
time     = 0  # Beat inicial
tempo    = 150  # BPM (beats/pulsações por minuto)
volume   = 100  # 0-127, (padrão MIDI)
arquivo  = "Questão1.mid"
# --------------------------

# Criar um objeto tipo Midi
MyMIDI = MIDIFile(1)  # Só um track
# No track 0 (o único neste caso), a partir do beat 0 (o primeiro) 
# o tempo será 60 (60 beats por segundo). 
MyMIDI.addTempo(track, time, tempo)

# Escala de Dó maior (4ta oitava)
escalaC=list(itertools.accumulate([2,2,1,2,2,2,1]))
Nota=NoteToMidi("C4")
Escala = [Nota if i == 0 else Nota+escalaC[i] for i in range(len(escalaC))]
escalaC=['C4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5']
# Converter a valores midi
for i, n in enumerate(Escala):
    print(n)
    # Parâmetros: faixa, canal, nota, tempo, duração, volume
    MyMIDI.addNote(track, channel, n, i, 1, volume)
    
    # Note que i toma os valores 0,1,2,3... Isso significa que, 
    # em cada pulsação teremos uma nota. Como a duração é 1 e o tempo é 60 (beats por minuto), 
    # cada nota terá uma duração de um segundo. 

# Escrever a um arquivo
with open(arquivo, "wb") as output_file:
    MyMIDI.writeFile(output_file)
    
    

60
64
65
67
69
71
72


# Questão 2

In [211]:
from midiutil import MIDIFile

# --------------------------
track    = 0  #faixa
channel  = 0  # canal
time     = 0  # Beat inicial
tempo    = 120  # BPM (beats/pulsações por minuto)
volume   = 100  # 0-127, (padrão MIDI)
arquivo  = "Questão2.mid"
# --------------------------

# Criar um objeto tipo Midi
MyMIDI = MIDIFile(1)  # Só um track
# No track 0 (o único neste caso), a partir do beat 0 (o primeiro) 
# o tempo será 60 (60 beats por segundo). 
MyMIDI.addTempo(track, time, tempo)

escalaMaior=list(itertools.accumulate([2,2,1,2,2,2,1]))
escalaMenor=list(itertools.accumulate([2,1,2,2,1,2,2]))
Nota=NoteToMidi("E4")
EscalaMaior = [Nota if i == 0 else Nota+escalaMaior[i] for i in range(len(escalaMaior))]
EscalaMenor = [Nota if j == 0 else Nota+escalaMenor[j] for j in range(len(escalaMenor))]

Escala=list(itertools.chain(EscalaMaior,EscalaMenor))
Escala.extend(Escala[::-1])
# Converter a valores midi
for i, n in enumerate(Escala):
    MyMIDI.addNote(track, channel, n, i, 1, volume)
    
    # Note que i toma os valores 0,1,2,3... Isso significa que, 
    # em cada pulsação teremos uma nota. Como a duração é 1 e o tempo é 60 (beats por minuto), 
    # cada nota terá uma duração de um segundo. 

# Escrever a um arquivo
with open(arquivo, "wb") as output_file:
    MyMIDI.writeFile(output_file)
    
    

# Questão 3

In [464]:
from collections import namedtuple
# Por exemplo NStr("C#4", 2)
NStr = namedtuple("NStr", "nota dur") 
# Por exemplo NNum(60, 2)
NNum = namedtuple("NNum", "nota dur") 

def toNum(seq):
    '''Transforma uma sequência de NStr para NNum '''
    return map(lambda x: NNum(NoteToMidi(x.nota), x.dur), seq) 

In [471]:
def GeradorEscalaMaior(nota):
    escalaMaior=list(itertools.accumulate([2,2,1,2,2,2,1]))
    Nota=NoteToMidi(nota)
    return [Nota if i == 0 else Nota+escalaMaior[i] for i in range(len(escalaMaior))]
EscalaMaior=GeradorEscalaMaior("C4")

a = 1
r = 2
length = 7

geometric = [a * r ** (n - 1) for n in range(1, length + 1)]
geometric1 = [4 * (1/2) ** (n - 1) for n in range(1, length + 1)]


j=0
EscalaAjustada=[]
for i in range(len(geometric)):
        while j<geometric[i]:
            EscalaAjustada.append(NNum(MidiToNote((EscalaMaior[i])),geometric1[i]))
            j=j+1
        j=0

In [472]:
def tocar(seq, arquivo, tempo=60, volume = 100, track=0, channel=0):
    '''Cria um arquivo midi com os valores da sequência (de objetos tipo NNum)'''
    MyMIDI = MIDIFile(1)  # Só um track
    MyMIDI.addTempo(0, 0, tempo)
    t = 0
    for n,d in seq:
        MyMIDI.addNote(track, channel, n, t, d, volume)
        t+=d

    with open(arquivo, "wb") as output_file:
        MyMIDI.writeFile(output_file)

tocar(toNum(EscalaAjustada),"Questão3.mid")

# Questão 4

In [473]:
import random
def randomseq():
    '''Gera uma sequência aleatória de bools'''
    while True:
        yield random.choice([True, False])

In [572]:
notas = [NStr(nota='G4', dur=2), NStr(nota='E4', dur=1), NStr(nota='E4', dur=1), 
         NStr(nota='F4', dur=2), NStr(nota='D4', dur=1), NStr(nota='D4', dur=1), 
         NStr(nota='C4', dur=1), NStr(nota='D4', dur=1), NStr(nota='E4', dur=1), NStr(nota='F4', dur=1), 
         NStr(nota='G4', dur=4), 
         NStr(nota='G4', dur=1), NStr(nota='G4', dur=1), NStr(nota='E4', dur=1), NStr(nota='E4', dur=1), 
         NStr(nota='F4', dur=1), NStr(nota='F4', dur=1), NStr(nota='D4', dur=1), NStr(nota='D4', dur=1), 
         NStr(nota='C4', dur=2), NStr(nota='D4', dur=2), 
         NStr(nota='C4', dur=4)]

In [605]:
note,time=list(zip(*list(notas)))

In [606]:
note=list(note)
time=list(time)

In [607]:
Hora=[time[i]/2 if next(randomseq())==True else time[i] for i in range(len(note))]

In [613]:
notas = [NStr(note[i],Hora[i]) for i in range(len(note))]

In [617]:
tocar(toNum(notas),"Questão4.mid")